# Importing Libraries

In [42]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri 
from sagemaker.session import s3_input, Session

import pandas as pd
import urllib
import numpy as np
import os
from sagemaker.predictor import csv_serializer
from sagemaker.serializers import CSVSerializer

# Setting region and unique bucket name

In [7]:
bucket_name = 'bankproject879' 
my_region = boto3.session.Session().region_name 
print(my_region)

us-east-1


# Creating S3 bucket 

In [8]:
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
        s3.create_bucket(Bucket=bucket_name)
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 bucket created successfully


# Setting an output path where the trained model will be saved

In [10]:
prefix = 'xgboost-as-a-built-in-algo'
output_path ='s3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://bankproject879/xgboost-as-a-built-in-algo/output


# Downloading Dataset and loading in dataframe

In [12]:
try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error: ',e)

try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


# Creating Train Test split

In [14]:
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


# Trasforming the training data and exporting as csv

In [16]:
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)

# Exporting data in to S3 bucket and importing again for training

In [21]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')

s3_input_train = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

# Test Data

In [44]:
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)

boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')

s3_input_test = sagemaker.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


# Importing Model

In [31]:
container = get_image_uri(boto3.Session().region_name,
                          'xgboost', 
                          repo_version='1.5-1')
     

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


# Initializing hyper parameters

In [32]:
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

# Construct a SageMaker estimator that calls the xgboost-container

In [34]:
estimator = sagemaker.estimator.Estimator(image_uri=container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          train_instance_count=1, 
                                          train_instance_type='ml.m5.2xlarge', 
                                          train_volume_size=5, # 5 GB 
                                          output_path=output_path,
                                          train_use_spot_instances=True,
                                          train_max_run=300,
                                          train_max_wait=600)

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_run has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_use_spot_instances has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_wait has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_volume_size has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


# Model fit

In [35]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2023-02-24-00-27-46-509


2023-02-24 00:27:46 Starting - Starting the training job...
2023-02-24 00:28:07 Starting - Preparing the instances for training......
2023-02-24 00:28:54 Downloading - Downloading input data...
2023-02-24 00:29:30 Training - Training image download completed. Training in progress...[2023-02-24 00:29:53.177 ip-10-0-177-245.ec2.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2023-02-24:00:29:53:INFO] Imported framework sagemaker_xgboost_container.training
[2023-02-24:00:29:53:INFO] Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
[2023-02-24:00:29:53:INFO] No GPUs detected (normal if no gpus installed)
[2023-02-24:00:29:53:INFO] Running XGBoost Sagemaker in algorithm mode
[2023-02-24:00:29:53:INFO] Determined delimiter of CSV input is ','
[2023-02-24:00:29:53:INFO] Determined delimiter of CSV input is ','
[2023-02-24:00:29:53:INFO] files path: /opt/ml/input/data/train
[2023-02-24:00:29:53:INFO] Determined delimiter of CS

# Deployment

In [36]:
xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2023-02-24-00-40-02-294
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2023-02-24-00-40-02-294
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2023-02-24-00-40-02-294


--------!

# Predictions

In [51]:
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array

# xgb_predictor.content_type = 'text/csv' # set the data type for an inference

xgb_predictor.serializer = csv_serializer # set the serializer type

predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!

predictions_array = np.fromstring(predictions[1:], sep=' ') # and turn the prediction into an array

print(predictions_array.shape)

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


(12357,)


# Calculating values for confucion matrix

In [53]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])

tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100

In [54]:
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.7%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10785)    34% (151)
Purchase        9% (1124)     66% (297) 



# Deleting endpoints and bucket data

In [55]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker:Deleting endpoint with name: sagemaker-xgboost-2023-02-24-00-40-02-294


[{'ResponseMetadata': {'RequestId': 'GJ9YSAV0CSN9SEGN',
   'HostId': 'buAq0VS6HfGXz9WEPCJZellmVQfjK7KHk+UNWJfmYG5P4M0r3lLMec6PKO3SDMDsPsO/HY3+Zxs=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'buAq0VS6HfGXz9WEPCJZellmVQfjK7KHk+UNWJfmYG5P4M0r3lLMec6PKO3SDMDsPsO/HY3+Zxs=',
    'x-amz-request-id': 'GJ9YSAV0CSN9SEGN',
    'date': 'Fri, 24 Feb 2023 01:05:33 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2023-02-24-00-27-46-509/profiler-output/system/incremental/2023022400/1677198540.algo-1.json'},
   {'Key': 'xgboost-as-a-built-in-algo/test/test.csv'},
   {'Key': 'xgboost-as-a-built-in-algo/train/train.csv'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2023-02-24-00-27-46-509/profiler-output/system/incremental/2023022400/1677198600.algo-1.json'},
   {'Key': 'xgboost-as-a-